# Annual outlet glacier terminus positions in northwestern Greenland


# Abstract
`...`

In [ ]:
# Import modules
import os
import geopandas as gpd
import pandas as pd
import manage
import glaciermetrics as gm
import matplotlib.pyplot as plt
import gplots as gp

In [ ]:
# User-specified parameters

# Relative path for file geodatabase containing glacier information
fgdb = '/gis-data/nwgreenland_decadal.gdb'

# File geodatabase layer names
termini_layer = 'annual_termini'
points_layer = 'glacier_points'
boxes_layer = 'glacier_boxes'

# Relative path to save analysis output
outpath = '/analysis/'

# Other data files
mask_file = '/gis-data/GimpOceanMask_90m/GimpOceanMask_90m.shp'

In [ ]:
# Load files
cwd = os.getcwd()

termini = gpd.read_file(cwd+fgdb, layer=termini_layer, driver='FileGDB')

points = gpd.read_file(cwd+fgdb, layer=points_layer, driver='FileGDB')
points.sort_values(by='Glacier_ID', inplace=True)
points.set_index('Glacier_ID', drop=False, inplace=True)

refboxes = gpd.read_file(cwd+fgdb, layer=boxes_layer, driver='FileGDB')
refboxes.sort_values(by='Glacier_ID', inplace=True)
refboxes.set_index('Glacier_ID', drop=False, inplace=True)

gimp = gpd.read_file(cwd+mask_file)
gimp.to_crs(epsg=3574)

In [ ]:
# Get scope of glaciers and time

GIDS = points.Glacier_ID.values

termini['Year'] = pd.to_datetime(termini['Year'], format='%Y')
YEAR_START = termini.Year.min().year
YEAR_END = termini.Year.max().year
YEARS = range(YEAR_START, YEAR_END+1)

DATE_START = termini.Source_Date.min()
DATE_END = termini.Source_Date.max()

In [ ]:
# Initialize dictionary of Glacier objects to store glacier information

all_glaciers = {id: manage.Glacier(id) for id in GIDS}
for id in all_glaciers:
    all_glaciers[id].refbox = refboxes.loc[id].geometry
    all_glaciers[id].officialname = points.loc[id].Official_Name
    all_glaciers[id].greenlandicname = points.loc[id].Greenlandic_Name
    all_glaciers[id].alternativename = points.loc[id].Alternative_Name
    

In [ ]:
# Construct an observation time series for each glacier

for id in GIDS:

    # Get reference line and all observations for glacier ID
    glacier = termini.query('Glacier_ID == @id')
    refbox = all_glaciers[id].refbox

    # Loop through all observations and process data
    for n in range(len(glacier)):
        observation = glacier.iloc[n]

        # Create a Terminus Observation for a row in geodataframe
        obs = manage.TerminusObservation(gid=int(observation.Glacier_ID),
                                         qflag=observation.Quality_Flag,
                                         imageid=observation.Image_ID,
                                         sensor=observation.Sensor,
                                         date=observation.Source_Date,
                                         year=observation.Year,
                                         geometry=observation.geometry,
                                         termination='',
                                         circadiandate='',
                                         season='')
        
        # Calculate glacier area against reference box
        obs.area = gm.glacierArea(obs, refbox)

        # Add glacier observation to time series
        all_glaciers[id].add_observation(obs)

    # Ensure that all observations are sorted by date
    all_glaciers[id].sort_by_date()

    # Extract time series of area and dates from all observations
    all_glaciers[id].areas = all_glaciers[id].extract('area')
    all_glaciers[id].dates = all_glaciers[id].extract('date')
    all_glaciers[id].hydroyears = all_glaciers[id].extract('hydroyear')
    all_glaciers[id].dayofhydroyear = all_glaciers[id].extract('dayofhydroyear')

    # Interpolate missing data between observations of area, term. area, length
    all_glaciers[id].interpareas = all_glaciers[id].interpolateMeasurements('area', YEARS)
    all_glaciers[id].interptermareas = all_glaciers[id].interpolateMeasurements('termarea', YEARS)
    all_glaciers[id].interplengths = all_glaciers[id].interpolateMeasurements('length', YEARS)
    all_glaciers[id].interpyears = all_glaciers[id].getInterpolatedYears('area', YEARS)

# 1. Introduction

```Outlet glacier discharge contributes to GrIS mass loss and acceleration (SLR). How much of GrIS mass loss is discharge vs. runoff? (this is why we care about terminus positions)```

```What are some proposed forcing mechanisms that control outlet glacier terminus position/change? How do they work? But we need to better understand terminus position changes over time in order to nail down forcing mechanisms.```

```Previous studies have looked at changes in terminus position over time, but they are generally sampling across either time or space, not both. References for NW GrIS - Twila, Michaela```

```In this paper we fill observational gap and sample across both space and time. Multidecadal annual record of terminus positions for 87 outlet glaciers in NW Greenland. Assess overall trends, regional variations in behavior, correlations with climatic factors.```

# 2. Data and Methods

We used both synthetic aperture radar (SAR) and optical satellite images to trace annual terminus positions for 87 outlet glaciers in northwestern Greenland (68.9&deg; N to 78.2&deg; N) from 1972 through 2019.

## 2.1 Satellite Images
We obtained terminus positions from SAR images taken by the Sentinel-1 and Radarsat-1 satellites for 11 of the 47 years in our record. These satellites use C-Band wavelengths, which allows them to image the surface regardless of clouds or darkness. For the most recent six years (2014-2015 through 2019-2020), we traced terminus positions in mosaics of contemporaneous Copernicus Sentinel-1A/B images (`Joughin et al., 2016`). These mosaics are typically created from images acquired in early February of their respective years and have 50-m resolution. Radarsat-1 mosaics were used to trace terminus positions for six winter (2000-2001, 2005-2009, 2012-2013) (`Joughin et al., 2017`; `Moon & Joughin, 2008`). These mosaics are formed from images collected from October through March and have nominal resolutions of 20 m.

We used imagery from all Landsat missions to map terminus positions for the remaining 36 years in our record, and for individual glaciers that were missing from or indiscernible in the SAR imagery. The majority of Landsat images used are from spring (March - May), although all months except January and December are represented. The resolution of these images ranges from 15 m (Landsat 8 panchromatic band) to 60 m (Landsat 1-5 Multispectral Scanner). We prescreened images in the USGS Global Visualization Viewer (GloVis) to confirm that glaciers of interest were not obscured by clouds and that the images were georeferenced well. Improper georeferencing can be an issue with older Landsat imges, so we compared our prospective images against known well-georeferenced images to ensure their accuracy. Due to winter darkness, it was not possible to select images from the same time of year as the SAR mosaics; instead, we chose images as close to winter as possible, with a preference for spring over fall to capture a more winter-like state to reduce the effects of seasonal variation. Because of the difficulty in finding sunlit, cloud-free imagery over each glacier, we could not acquire images at the same time every year for a given glacier, and for several years there was no viable imagery at all for some glaciers.

## 2.2 Terminus Positions
We used GIS software to manually trace annual terminus positions from satellite imagery for 87 outlet glaciers in northwestern Greenland from 1972 through 2019. This dataset builds on a preexisting dataset covering six winters between 2000 and 2012 (`Joughin et al., 2017`; `Moon & Joughin, 2008`). The study are ranges from Saqqarliup Sermia (68.9&deg; N, 50.3&deg; W; ~35 km southwest of Jakobshavn Isbr&aelig;) to Bamse Gletsjer (78.2&deg; N, 72.7&deg; W) ([Fig. 1](#Figure-1)). We selected marine-terminating outlet glaciers that are at least ~1.5 km wide at the terminus and are flowing at least ~1000 m a&#8315;&sup1;. We traced each glacier's terminus position once per hydrological year (September 1 through August 31 (`Ettema et al., 2009`)) in every year for which suitable imagery was available. We used winter or near-winter imagery whenever possible, although for some glaciers and years, suitable imagery was only available in the summer ([Fig. 2a](#Figure-2a)).

#### Figure 1
**Northwest Greenland study area**
* `include glacier zones that are used in ancillary data`
* `also include velocity maps as part of the location map - there's a 20-year one in the gimp dataset that is pretty clean`

In [ ]:
fig = plt.figure(figsize=(10,3))
ax = fig.add_axes([0., 0., 1., 1.])
ax.set_aspect('equal')

# Plot Greenland outline
gimp.plot(ax=ax, linewidth=0.2, color='silver')

# Plot glacier points on Greenland outline
points.plot(ax=ax, markersize=3, color='teal', linewidth=0.2, edgecolor='black')

# Set map boundaries based on observed region
bounds = points.geometry.bounds
bound_buffer = 50000
ax.set_xlim([bounds.minx.min()-bound_buffer, bounds.maxx.max()+bound_buffer])
ax.set_ylim([bounds.miny.min()-bound_buffer, bounds.maxy.max()+bound_buffer])
ax.axis('off')

# plt.savefig('{}NWGr_study_area.png'.format(outpath), \
    # bbox_inches='tight')

Errors in terminus position may arise from the imagery used and digitization errors. The primary sources of image errors are orthorectification and georeferencing. To reduce such errors, candidate images were compared with control images and discarded if they were noticeably offset or distorted. Digitization errors could derive from orthorectification, image resolution, manual tracing errors, or uncertain terminus location in imagery (due to _e.g._ shadows or an indistinct transition from glacier to m\eacutelange) (`Joughin et al., 2017`). Uncertain terminus positions were flagged during tracing, and were compared with close-in-time images from the same or other satellite platforms when possible.

In addition to errors associated with digitization of the images, there is additional uncertainty introduced by sampling seasonal variability at different times of year. For the terminus positions mapped from Landsat imagery, it was not possible to get sunlit and cloud-free images over each glacier at the same time every year, and in some years there was no viable imagery ([Fig. 2b](#figure-2b)). This timing variation complicates the year-to-year comparison of glacier areas for any glaciers which might have seasonal variability in terminus position.
* `need some kind of error assessment. Refer to Twila work on errors (quantification)`
* `address season issue with monthly positions to constrain annual variability`

#### Figure 2a 
**Seasonal distribution of terminus observations**
* `plot as seasonal distribution, or as DOY?`

#### Figure 2b

**Terminus records**: annual observations (filled circle) and interpolations (empty circle) for each glacier. Red line indicates first year in which all glaciers have either an observed or interpolated measurement.

- `TODO: fix to not interpolate for 2020`

In [ ]:
fig = plt.figure(figsize=(10, 7))
gp.annualObservations(fig, all_glaciers, YEARS, show_firstyear=True)
# plt.savefig('{}observation_timeseries.png'.format(outpath), \
    # bbox_inches='tight')

## 2.3 Glacier Area Change
We calculated glacier area change over time using the box method (`Moon & Joughin, 2008`). Each glacier has a static, open-ended rerference box that approximately delineates the main region of ice flow. The box sides are roughly parallel to ice flow, and the box 'back' is perpendicular to ice flow at an arbitrary location up-glacier of the extent of maximum observed terminus retreat. Where a terminus trace intersects the open end of the box, a polygon is formed, and the area of that polygon represents the glacier's reference area at that point in time. Repeating this process for each terminus trace for a glacier forms a time series of reference areas, from which we determine the annual area change. While the areas are arbitrarily determined by the box size, the area differences between successive terminus traces represent the annual gain or loss of area.

Errors in the area change calculation may come from inacurrate and missing terminus observations, and from possible seasonal variability of terminus locations. In years where no observations were made, we linearly interpolated between the prior and subsequent observations to estimate glacier areas during the missing years ([Fig. 2b](#figure-2b)). For glaciers with missing observations in the beginning of the record, we did not interpolate prior to the first observation in the record.

```Another source of error might be seasonal variability. Do this analysis... Note that seasonal variability could provide some short-term variability but it should even out over longer periods.```
```Say something about how focusing on area change removes the arbitrariness of how the box is drawn. It's hard to compare glaciers of different sizes; doesn't account for terminus width vs area change (look at length changes). Check that all traces were converted to areas.```

In order to compare area change trends between glaciers of different sizes, we also determined the percent area change over time for each glacier. In an individual glacier area time series, we set the minimum glacier area as 0% and the maximum glacier area as 100%, and linearly scaled the other measured areas between those set points. This method places every glacier on the same 0-100% scale and excludes the arbitrariness of the glacier box size, allowing straightforward comparison of glacier area changes. `MOVE TO RESULTS!` `Make clear the purpose of doing this - to show timing similarities for different glaciers without the changes being swamped by large glaciers. Some glaciers tend to dominate; to look at individual glaciers we did this...`

# 3. Results
`...`

## 3.1 Terminus Positions
We created a dataset of `3427` annual terminus positions for our 87 selected glaciers from 1972 through 2019. The median number of annual observations per glacier is 39, and nearly all glaciers were observed in 36 to 44 of the 47 years examined. Only three glaciers have fewer (31-32) observations; these glaciers are located just south of Thule Air Force Base and have limited imagery available in the 1980s and early 1990s. After interpolating area changes between glacier observations, the first year with either an observation or an interpolation available for each glacier is 1977-1978. Therefore, in this paper our analyses start in this year except where noted.

` Figure 3 - area, length, normalized`

## 3.2 Ancillary Climate Data
`...`

### 3.2.1 Surface Mass Balance
` - look at melt or runoff directly`
` - more melt occurring... hydrofracture?`
` - 

### 3.2.2 Ocean Temperature
`...`
` - look at Ecco?`
` - plot the variance with the data`

### 3.2.3 Sea Ice Concentration
` - annual average as line plot`
` - length of sea ice season`

### 3.2.4 Sea Surface Temperature
`...`

# 4. Discussion

# 5. Conclusions

# 6. Data Sources

Sentinel-1: European Space Agency, Copernicus Program. Mosaics from NSIDC/MEaSUREs/GIMP [https://nsidc.org/data/nsidc-0723/versions/2](https://nsidc.org/data/nsidc-0723/versions/2).

Radarsat-1: Canadian Space Agency. Terminus traces from Joughin et al. (2017) and Moon & Joughin (2008).

Landsat 1-5, 7, 8: NASA/USGS, Landsat Program. Images identified with USGS GloVis [http://glovis.usgs.gov/](http://glovis.usgs.gov/) and downloaded from Google Cloud Platform [https://console.cloud.google.com/storage/browser/gcp-public-data-landsat](https://console.cloud.google.com/storage/browser/gcp-public-data-landsat).

RACMO – unpublished data, Brice Noël

Ocean temperatures: ICES Dataset on Ocean Hydrography (ICES, 2014) [http://www.ices.dk/marine-data/data-portals/Pages/ocean.aspx](http://www.ices.dk/marine-data/data-portals/Pages/ocean.aspx)

Sea ice concentration: NOAA/NSIDC Climate Data Record of Passive Microwave Sea Ice Concentration, Version 3 [https://nsidc.org/data/g02202](https://nsidc.org/data/g02202).
